In [31]:
import lxml.html as lx
import pandas as pd
import numpy as np
import requests
import requests_cache
import numpy as np
import nltk
import nltk.corpus
from nltk.corpus import wordnet
from itertools import chain
from plotnine import *
import time
import matplotlib.pyplot as plt
import argparse
%matplotlib inline

In [32]:
base_url = "https://boards.na.leagueoflegends.com/en/c/player-behavior-moderation"
def get_html(base_url):
    requests_cache.install_cache("project")
    response = requests.get(base_url)
    response.raise_for_status
    response = response.text
    html = lx.fromstring(response)
    html.make_links_absolute(base_url)
    return html

#BEFORE GOING INTO THE URL
#title
html = get_html(base_url)
html.xpath("//*[contains(@class, 'title-span')]")[0].text_content()

#region
html.xpath("//*[contains(@class, 'realm')]")[0].text_content()

#url
url = html.xpath("//*[contains(@class, 'title-link')]/@href")

#paragraphs of player issues
test = requests.get(url[4])
test = test.text
test = lx.fromstring(test)
test.xpath("//*[contains(@id, 'content')]/div")[0].text_content()

#paragraphs of player issues answered

'\n          Hello everyone,\r\n\r\nI have been playing league of legends for many years on and off. I have fun playing it, most of the time (i like to win, who doesnt) which is why i always get back on. But ive had one of the worst experiences ever playing it not just yesturday, but today also. From the same 2 people too.. I have NEVER seen this type of behavior before. \r\n\r\nThese 2 people were on enemy team, as soon as the game started, they started spamming all chat being downright condescending and saying things like "you\'re an egirl u suck, we so got this win easy, and so on..." and minions hadnt even spawned yet. Today same thing with the same 2 guys, even though i reported them yesturday and also today. I was so chocked by their behavior that i didnt even know what to say, it was just unbelievable.\r\n\r\nLike i mentioned though in the past years i have NEVER seen this. Players tend to mostly critisize skill or champ pick or items whatever. But never have i seen someone down

**The below functions get all the url in the league of legends player behaviour forum. They only showed the most recent ones up to 1000 urls, dating back 4 months ago**

In [33]:
def player_behavior_url(base_url, forum):
    requests_cache.install_cache('player_logs')
    end = False
    base_url = "https://boards.na.leagueoflegends.com"
    url = []
    start = 0 
    while True:
        try:
            time.sleep(.1)
            response = requests.get(forum+str(start))
            response.raise_for_status()
            result = response.json()
            result = result['discussions']
            html = lx.fromstring(result)
            html.make_links_absolute(base_url)
            url += html.xpath("//tr//@data-href")
            start += 50;
        except:
            break
    return url

base_url = "https://boards.na.leagueoflegends.com/"
extension = "api/ZGEFLEUQ/discussions?sort_type=hot&num_loaded="
url = player_behavior_url(base_url, base_url+extension)
url

['/en/c/player-behavior-moderation/lUbMFwMZ-ten-reasons-why-its-better-not-to-flame-back',
 '/en/c/player-behavior-moderation/OykwtTZu-a-player-that-went-015-admitted-to-troll-in-chat-is-still-playing',
 '/en/c/player-behavior-moderation/sERb1G27-tips-for-helping-keep-your-cool',
 '/en/c/player-behavior-moderation/j4V309e4-perma-banned',
 '/en/c/player-behavior-moderation/U7N6a5dv-why-soft-inting-is-hardly-punished',
 '/en/c/player-behavior-moderation/sxdwaRNs-soft-inting-trolling-goes-by-without-punishment',
 '/en/c/player-behavior-moderation/HGQTF1i8-how-is-this-okay',
 '/en/c/player-behavior-moderation/WRzyJa8A-not-the-normal-raging',
 '/en/c/player-behavior-moderation/3ATEdyM5-instant-feedback-report',
 '/en/c/player-behavior-moderation/nUJPBxza-just-a-rant-hopefully-someone-is-listening',
 '/en/c/player-behavior-moderation/161jQpB5-tilt-i-need-help-im-desperate',
 '/en/c/player-behavior-moderation/AnKKjbN0-if-someone-goes-past-015-it-should-flag-their-account',
 '/en/c/player-beha

How to connect git through 3 people?
How to go to next page/ show more?

In [35]:
def get_page_url(base_url, page_url):
    requests_cache.install_cache('page_logs')
    next_page_available = True
    cur_page = 1;
    urls = []
    while True:
        try:
            time.sleep(.1)
            response = requests.get(page_url+str(cur_page))
            html = lx.fromstring(response.text)
            html.make_links_absolute(base_url)
            url = html.xpath("//tbody//td/a/@href")
            urls += url
            cur_page +=1;
        except:
            break
    return urls

base_url = "http://forums.na.leagueoflegends.com/"
extension = "board/forumdisplay.php?f=59&page="
pages_url = get_page_url(base_url, base_url+extension)

In [36]:
def unique(links):
    url = []
    for items in links:
        if items not in url:
            url.append(items)
    return url
pages_url = unique(pages_url)

In [110]:
#Other website for the name and the past rank
op_url = "https://na.op.gg/summoner/userName="

def rank_lvl(name):
    requests_cache.install_cache("opna")
    time.sleep(.1)
    response = requests.get(op_url+name)
    response.raise_for_status()
    html = lx.fromstring(response.text)
    
    lvl = html.xpath("//*[contains(@class,'Level tip')]")
    recent_rank = html.xpath("//*[contains(@class, 'PastRankList')]/li[last()]")
    if len(lvl) > 0:
        lvl = lvl[0].text_content().strip()
    
    if len(recent_rank) > 0:
        recent_rank = recent_rank[0].text_content().strip()
        
    if len(lvl) > 0 and len(recent_rank) >0:
        return recent_rank,lvl
    
    elif len(lvl)>0:
        return np.nan,lvl
    
    else:
        return np.nan, np.nan

(nan, '30')

Attempting to extract user information: name, date, questions, answers

In [ ]:
def page_info(url):
    dates = []
    names = []
    members = []
    texts = []
    lvl = []
    rank = []
    requests_cache.install_cache("page_posts")
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = lx.fromstring(response.text)
        time.sleep(.1)
        title = html.xpath("//*[contains(@class, 'float-left')]/h3")[0].text_content()
        all_dates = html.xpath("//*[contains(@class, 'post-header padding-side-10 clearfix')]")
        all_users = html.xpath("//*[contains(@class, 'post-user')]")
        member_type = html.xpath("//*[contains(@class, 'post-col-left')]//p")[1::2]
        for i in range(len(all_dates)):
            dates.append(all_dates[i].text_content().strip())
            names.append(all_users[i].text_content().strip())
            members.append(member_type[i].text_content().strip())
            texts.append(html.xpath("//*[contains(@class,'post-message')]//p")[i].text_content().strip())
        for name in names:
            op_na = rank_lvl(name)
            rank.append(op_na[0])
            lvl.append(op_na[1])
        return pd.DataFrame({'user_name':names,"tier": rank, "level": lvl, "tier": rank, "level": lvl, 'member_type': members, 'dates': dates,'title': title, 'text':texts})
    except:
        return

page_df = [page_info(url) for url in pages_url]

In [65]:
df = pd.concat(page_df, axis=0).reset_index()
df.columns.values[0] = ['tree']